## Discrete Search Spaces

In [90]:
from overrides import overrides
import numpy as np
from typing import Tuple, List, Optional
from archai.discrete_search import ArchaiModel, DiscreteSearchSpace

Discrete search spaces in Archai are defined using the `DiscreteSearchSpace` abstract class:

```python

class DiscreteSearchSpace(EnforceOverrides):

    @abstractmethod
    def random_sample(self) -> ArchaiModel:
        ...
        
    @abstractmethod
    def save_arch(self, model: ArchaiModel, path: str) -> None:
        ...

    @abstractmethod
    def load_arch(self, path: str) -> ArchaiModel:
        ...

    @abstractmethod
    def save_model_weights(self, model: ArchaiModel, path: str) -> None:
        ...

    @abstractmethod
    def load_model_weights(self, model: ArchaiModel, path: str) -> None:
        ...
```

### CNN Search Space Example

Let's start with a simple search space for image classification

In [40]:
import torch
from torch import nn


class MyModel(nn.Module):
    def __init__(self, nb_layers: int = 5, kernel_size: int = 3, hidden_dim: int = 32):
        super().__init__()
        
        self.nb_layers = nb_layers
        self.kernel_size = kernel_size
        self.hidden_dim = hidden_dim
        
        layer_list = []

        for i in range(nb_layers):
            in_ch = (3 if i == 0 else hidden_dim)
            
            layer_list += [
                nn.Conv2d(in_ch, hidden_dim, kernel_size=kernel_size, padding='same'),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU()
            ]

        layer_list += [
            nn.Conv2d(hidden_dim, 1, kernel_size=1, padding='same'),
            nn.Sigmoid()
        ]
        
        self.model = nn.Sequential(*layer_list)
    
    def forward(self, x):
        return self.model(x)
    
    def get_archid(self):
        return f'({self.nb_layers}, {self.kernel_size}, {self.hidden_dim})'

In [41]:
m = MyModel(nb_layers=1)
m

MyModel(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (4): Sigmoid()
  )
)

In [42]:
m.get_archid()

'(1, 3, 32)'

Let's overide DiscreteSearchSpace

In [43]:
import json
from typing import Tuple
from random import Random

class CNNSearchSpace(DiscreteSearchSpace):
    def __init__(self, min_layers: int = 1, max_layers: int = 12,
                 kernel_list=(1, 3, 5, 7), hidden_list=(16, 32, 64, 128),
                 seed: int = 1):

        self.min_layers = min_layers
        self.max_layers = max_layers
        self.kernel_list = kernel_list
        self.hidden_list = hidden_list
        
        self.rng = Random(seed)
        
    @overrides
    def random_sample(self) -> ArchaiModel:
        # Randomly chooses architecture parameters
        nb_layers = self.rng.randint(self.min_layers, self.max_layers)
        kernel_size = self.rng.choice(self.kernel_list)
        hidden_dim = self.rng.choice(self.hidden_list)
        
        model = MyModel(nb_layers, kernel_size, hidden_dim)
        
        # Wraps model into ArchaiModel
        return ArchaiModel(arch=model, archid=model.get_archid())

    @overrides
    def save_arch(self, model: ArchaiModel, file: str):
        with open(file, 'w') as fp:
            json.dump({
                'nb_layers': model.arch.nb_layers,
                'kernel_size': model.arch.kernel_size,
                'hidden_dim': model.arch.hidden_dim
            }, fp)

    @overrides
    def load_arch(self, file: str):
        config = json.load(open(file))
        model = MyModel(**config)
        
        return ArchaiModel(arch=model, archid=model.get_archid())

    @overrides
    def save_model_weights(self, model: ArchaiModel, file: str):
        state_dict = model.arch.get_state_dict()
        torch.save(state_dict, file)
    
    @overrides
    def load_model_weights(self, model: ArchaiModel, file: str):
        model.arch.load_state_dict(torch.load(file))


In [44]:
ss = CNNSearchSpace(hidden_list=[32, 64, 128])

Sampling a model

In [45]:
m = ss.random_sample()
m

ArchaiModel(
	archid=(3, 1, 64), 
	metadata={}, 
	arch=MyModel(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (10): Sigmoid()
  )
)
)

Saving an architecture

In [46]:
ss.save_arch(m, 'arch.json')

In [47]:
!cat arch.json

{"nb_layers": 3, "kernel_size": 1, "hidden_dim": 64}

Loading an architecture (not the weights)

In [48]:
ss.load_arch('arch.json')

ArchaiModel(
	archid=(3, 1, 64), 
	metadata={}, 
	arch=MyModel(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(64, 1, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (10): Sigmoid()
  )
)
)

### Making the search space compatible with different types of algorithms

* Evolutionary-based algorithms:
 - User must subclass `EvolutionarySearchSpace` and implement `EvolutionarySearchSpace.mutate` and `EvolutionarySearchSpace.crossover`


* BO-based algorithms:
 - User must subclass `BayesOptSearchSpace` and override `BayesOptSearchSpace.encode`


#### Example

In [91]:
from archai.discrete_search import EvolutionarySearchSpace, BayesOptSearchSpace

In [107]:
class CNNSearchSpaceExt(CNNSearchSpace, EvolutionarySearchSpace, BayesOptSearchSpace):
    ''' We are subclassing CNNSearchSpace just to save up space'''
    
    @overrides
    def mutate(self, model_1: ArchaiModel) -> ArchaiModel:
        config = {
            'nb_layers': model_1.arch.nb_layers,
            'kernel_size': model_1.arch.kernel_size,
            'hidden_dim': model_1.arch.hidden_dim
        }
        
        if self.rng.random() < 0.2:
            config['nb_layers'] = self.rng.randint(self.min_layers, self.max_layers)
        
        if self.rng.random() < 0.2:
            config['kernel_size'] = self.rng.choice(self.kernel_list)
        
        if self.rng.random() < 0.2:
            config['hidden_dim'] = self.rng.choice(self.hidden_list)
        
        mutated_model = MyModel(**config)
        
        return ArchaiModel(
            arch=mutated_model, archid=mutated_model.get_archid()
        )
    
    @overrides
    def crossover(self, model_list: List[ArchaiModel]) -> ArchaiModel:
        model_1, model_2 = model_list[:2]
        
        new_config = {
            'nb_layers': self.rng.choice([model_1.arch.nb_layers, model_2.arch.nb_layers]),
            'kernel_size': self.rng.choice([model_1.arch.kernel_size, model_2.arch.kernel_size]),
            'hidden_dim': self.rng.choice([model_1.arch.hidden_dim, model_2.arch.hidden_dim]),
        }
        
        crossover_model = MyModel(**new_config)
        
        return ArchaiModel(
            arch=crossover_model, archid=crossover_model.get_archid()
        )
    
    @overrides
    def encode(self, model: ArchaiModel) -> np.ndarray:
        return np.array([model.arch.nb_layers, model.arch.kernel_size, model.arch.hidden_dim])

In [108]:
ss = CNNSearchSpaceExt(hidden_list=[32, 64, 128])

Example

In [109]:
m = ss.random_sample()
m.archid

'(3, 1, 64)'

In [110]:
ss.mutate(m).archid

'(8, 1, 64)'

In [111]:
ss.encode(m)

array([ 3,  1, 64])

Now `CNNSearchSpaceExt` is compatible with Bayesian Optimization and Evolutionary based search algorithms!

**To see a list of built-in search spaces, go to `archai/discrete_search/search_spaces`**

Example: 

In [120]:
from archai.discrete_search.search_spaces.segmentation_dag.search_space import SegmentationDagSearchSpace

ss = SegmentationDagSearchSpace(nb_classes=1, img_size=(64, 64), max_layers=3)
ss.mutate(ss.random_sample())

ArchaiModel(
	archid=74f66612a0d01c5b7d4702234756b0ee4ffa5abc_64_64, 
	metadata={'parent': '32fa5956ab3ce9e05bc42836599a8dc9dd53e847_64_64'}, 
	arch=SegmentationDagModel(
  (edge_dict): ModuleDict(
    (input-output): Block(
      (op): Sequential(
        (0): NormalConvBlock(
          (conv): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU()
        )
      )
    )
  )
  (stem_block): NormalConvBlock(
    (conv): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (up): Upsample(size=(64, 64), mode=nearest)
  (post_upsample): Sequential(
    (0): NormalConvBlock(
      (conv): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_st